## Code Interpreter

In [ ]:
from dotenv import load_dotenv  
import os  
# Configure logger
import logging
import sys
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import CodeInterpreterTool
from azure.ai.agents.models import ListSortOrder
from azure.monitor.opentelemetry import configure_azure_monitor

# Load the .env file  
load_dotenv(override=True) 

# Now you can use the environment variables, for example:  
project_endpoint = os.environ["PROJECT_ENDPOINT"]
model=os.environ["MODEL_DEPLOYMENT_NAME"]

print(project_endpoint + " " + model)


In [ ]:
# Create a logger
logger = logging.getLogger("notebook_logger")
logger.setLevel(logging.INFO)

# Clear existing handlers to avoid duplicate logs
if logger.hasHandlers():
    logger.handlers.clear()

# Create handler that outputs to notebook
handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

logger.addHandler(handler)

# Example usage
logger.info("Logger is now printing to the notebook!")

In [ ]:
# Retrieve the project endpoint from environment variables
project_endpoint = os.environ["PROJECT_ENDPOINT"]

# Initialize the AIProjectClient
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),
)

os.environ['AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED'] = 'true'
# Enable Azure Monitor tracing
application_insights_connection_string = project_client.telemetry.get_connection_string()

if not application_insights_connection_string:
    print("Application Insights was not enabled for this project.")
    print("Enable it via the 'Tracing' tab in your Azure AI Foundry project page.")
    exit()
    
configure_azure_monitor(connection_string=application_insights_connection_string)

In [ ]:

agent_name = os.environ["AZURE_AI_AGENT_NAME_03"]
agent_list = project_client.agents.list_agents()

agent = None
if agent_list:
    for agent_object in agent_list:
        if agent_object.name == agent_name:
            agent = agent_object

if not agent:
    print(f"Agent '{agent_name}' not found in the project.")

In [ ]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import CodeInterpreterTool
from azure.ai.agents.models import ListSortOrder

code_interpreter = CodeInterpreterTool()


In [ ]:
from azure.ai.agents.models import FilePurpose


# Create an Azure AI Client from an endpoint, copied from your Azure AI Foundry project.
# You need to login to Azure subscription via Azure CLI and set the environment variables
project_endpoint = os.environ["PROJECT_ENDPOINT"]  # Ensure the PROJECT_ENDPOINT environment variable is set

# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)

if not agent:
    FILE_PATH = "./data/nifty_500_quarterly_results.csv"  # Path to the CSV file you want to upload

    # Step 2: Upload the CSV file  
    print(f"Step 2: Uploading file '{FILE_PATH}'...")  
    uploaded_file = project_client.agents.files.upload_and_poll( 
                    file_path=FILE_PATH,  
                    purpose=FilePurpose.AGENTS  
                )  
    print(f"Uploaded file, file ID: {uploaded_file.id}") 

    code_interpreter = CodeInterpreterTool(file_ids=[uploaded_file.id])

    agent = project_client.agents.create_agent(
                    model=os.environ["MODEL_DEPLOYMENT_NAME"],
                    name=agent_name,
                    instructions="You are helpful agent that has the ability to write and execute code to analyze data.",
                    tools=code_interpreter.definitions,
                    tool_resources=code_interpreter.resources,
                )
else:
    logger.info(f"Found agent by name '{agent_name}', ID={agent.id}")

In [ ]:
thread = project_client.agents.threads.create()
message = project_client.agents.messages.create(
                thread_id=thread.id,
                role="user",
                content="Review the csv file, Which company has the highest net profit margin in Q1, and what does this indicate about its operational efficiency compared to others in the dataset?",
            )

run = project_client.agents.runs.create_and_process(
                thread_id=thread.id,
                agent_id=agent.id)

In [ ]:
print(thread.id)
print(agent.id)
print(run.id)
print(run.status)

In [ ]:
run = project_client.agents.runs.get(
                thread_id=thread.id,
                run_id=run.id
            )

print("Run status:", run.status)

In [ ]:
import json
from IPython.display import display, Markdown


if run.status == "failed":
        print(f"Run failed: {run.last_error}")
else:
        messages = project_client.agents.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

for message in messages:
            if message.text_messages:
                markdown_text = f"{message.role}: {message.text_messages[-1].text.value}"
                display(Markdown(markdown_text))

In [ ]:
run_steps = project_client.agents.run_steps.list(thread_id=thread.id, run_id=run.id)

for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")


        print()  # add an extra newline between steps